In [14]:
import random
import numpy as np
import tensorflow as tf
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
import unidecode

In [15]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6120)])
#   except RuntimeError as e:
#     print(e)

In [16]:
lemmatizer=WordNetLemmatizer()
data = open("Json_detalle_conversaciones_bot.json").read()
intents = json.loads(data)
words = pickle.load(open('words_chatbot.pkl','rb'))
clases = pickle.load(open('clases_chatbot.pkl','rb'))
model = tf.keras.models.load_model('Chatbot_base_chatbot_protocolarv2.tf')

In [17]:
print(intents['intents'])

[{'tag': 'Identificacion', 'patterns': ['como te llamas', 'como puedo llamarte', 'cual es tu nombre'], 'responses': ['me llamo FINANCIALYTICS-BOT, tu bot de apoyo en Financialitica', 'Llámame FINANCIALYTICS-BOT,soy bot de apoyo en Financialitica']}, {'tag': 'Presentacion', 'patterns': ['hola', 'buenas', 'buenos dias', 'buenas tardes', 'hi', 'hello', 'morning'], 'responses': ['Buenas, ¿en qué puedo ayudarte?', 'Hola, ¿en qué puedo ayudarte?', '¿en qué puedo serte útil?', 'Buenas, requieres de algún apoyo con el Website o con algún Producto?']}, {'tag': 'Proposito', 'patterns': ['cual es tu proposito', 'como me puedes ayudar', 'que puedes hacer por mi'], 'responses': ['Soy un bot creado con Inteligencia Artificial, entrenado por el equipo de Financialitica, para responder  preguntas limitadas sobre funcionamiento del website y  productos .', 'Soy un bot entrenado por el equipo de Financialitica, que te va a servir de guia en el Website homónimo. Puedes preguntar sobre el tema que desees.

In [18]:
model.summary()

Model: "chatbot_trainer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  6912      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  8256      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  520       
                                                                 
Total params: 15,688
Trainable params: 15,688
Non-t

In [19]:
# model.predict([])

In [20]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize((unidecode.unidecode(word)).lower())
                      for word in sentence_words]
    return sentence_words



In [21]:
# this function will return 1 if the word is contained in our sentence and in the list previously defined 
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [22]:
# This function predicts a cleansed data with a mark in the bow (flag = 1 )
# It also appends the class - tag in the clases list where it belongs 
def predict_clases(sentence,model):
    print(clean_up_sentences(sentence))
    p = bow(sentence,words,show_details=True)
    print(p)
    res = model.predict(np.array([p]))[0]
    
    ERROR_THRESHOLD=0.3#0.25
    results = [[ i , r ] for i , r  in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key = lambda x : x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({
            'intent' : clases[r[0]],
            'probability': str(r[1])

        } )
        return return_list


In [23]:
# this function will print a random response in the class according the input that the user made
def get_response(intents_list,intents_json):
    print('intents_list',intents_list)
    tag = intents_list[0]['intent']
    list_of_intents = intents_json ['intents']
    result = ""
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print ("Esta vivo!")

Esta vivo!


In [24]:


# while True:
#     message = ""
#     ints = predict_clases(message)
#     res = get_response(ints , intents)
#     print(res)

#     # if message == 'salir':
    #     C=False


In [25]:
# message = "whats your name?"

# ints = predict_clases(message,model)
# res = get_response(ints , intents)
# print(res)

In [28]:
message = "adios"
print("esto es message",message)
print("esto es message",model)

ints = predict_clases(message,model)
print("esto es ints",ints)
res = get_response(ints , intents)
print(res)

esto es message adios
esto es message <keras.saving.legacy.saved_model.load.Chatbot_trainer object at 0x7f15e87bbe80>
['adios']
found in bag: adios
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 18ms/step
esto es ints [{'intent': 'Despedida', 'probability': '1.0'}]
intents_list [{'intent': 'Despedida', 'probability': '1.0'}]
Ha sido un placer ayudarte, hasta la próxima!
